# Preparing the Data

In [1]:
# import resources
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
def read_conllu_file(filename):
    index = []
    words = []
    pos_tags = []
    word_sentence = []
    pos_tags_sentences = []
    index_sentence = []
    words_total = []
    pos_tags_total = []

    count = 0
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            if line.strip():
                if line.startswith("#"):
                    continue
                fields = line.strip().split("\t")
                if "." in fields[0]:
                    continue
                index_sentence.append(fields[0])
                word_sentence.append(fields[1])
                pos_tags_sentences.append(fields[3])
                words_total.append(fields[1])
                pos_tags_total.append(fields[3])
            else:
                index.append(index_sentence)
                print
                pos_tags.append(pos_tags_sentences)
                words.append(word_sentence)
                word_sentence = []
                pos_tags_sentences = []
                index_sentence = []
#             print ( fields[0])
    return index, words, pos_tags, words_total, pos_tags_total

In [3]:
index, words, pos_tags, words_total, pos_tags_total= read_conllu_file("/kaggle/input/datasent-for-pos-tagging-iiith/en_atis-ud-train.conllu")
index_dev, words_dev, pos_tags_dev, words_total_dev, pos_tags_total_dev= read_conllu_file("/kaggle/input/datasent-for-pos-tagging-iiith/en_atis-ud-dev.conllu")
index_test, words_test, pos_tags_test, words_total_test, pos_tags_total_test= read_conllu_file("/kaggle/input/datasent-for-pos-tagging-iiith/en_atis-ud-test.conllu")


In [4]:
print(words_dev[0])

['i', 'would', 'like', 'the', 'cheapest', 'flight', 'from', 'pittsburgh', 'to', 'atlanta', 'leaving', 'april', 'twenty', 'fifth', 'and', 'returning', 'may', 'sixth']


In [5]:
num_sentences = len(index)
num_words = len(set(words_total))
num_tags = len(set(pos_tags_total))
print( " Total number of sentences ", num_sentences)
print( " Total number of unique words ", num_words)
print( " Total number of tags ", num_tags)
# print( set ( pos_tags))

 Total number of sentences  4274
 Total number of unique words  863
 Total number of tags  13


In [6]:
num_sentences_dev = len(index_dev)
num_words_dev = len(set(words_total_dev))
num_tags_dev = len(set(pos_tags_total_dev))
print( " Total number of sentences ", num_sentences_dev)
print( " Total number of unique words ", num_words_dev)
print( " Total number of tags ", num_tags_dev)
print(set(pos_tags_total_dev))

 Total number of sentences  570
 Total number of unique words  448
 Total number of tags  13
{'ADV', 'PART', 'ADJ', 'INTJ', 'NOUN', 'ADP', 'VERB', 'AUX', 'PROPN', 'NUM', 'CCONJ', 'PRON', 'DET'}


In [7]:
num_sentences_test = len(index_test)
num_words_test = len(set(words_total_test))
num_tags_test = len(set(pos_tags_total_test))
print( " Total number of sentences ", num_sentences_test)
print( " Total number of unique words ", num_words_test)
print( " Total number of tags ", num_tags_test)

 Total number of sentences  586
 Total number of unique words  476
 Total number of tags  13


# Encoding and Embedding

In [8]:
import torchtext
from torchtext.vocab import Vocab
from collections import Counter

In [9]:
word_to_idx = {}
idx_to_word = {}

unique_words = set(words_total)

# print( unique_words)
for i, word in enumerate(unique_words):
    word_to_idx[word] = i
    idx_to_word[i] = word
    
X = []
for sent in words:
#     print(sent)
    encoded_sent = [word_to_idx[word] for word in sent]
    X.append(encoded_sent)
    
X_test = []
for sent in words_test:
    encoded_sent = []
    for word in sent:
        if word in unique_words:
            encoded_sent.append(word_to_idx[word])
        else:
            encoded_sent.append(0)
    X_test.append(encoded_sent)

count = 0 
counta = 0 
X_dev = []
for sent in words_dev:
    encoded_sent = []
    for word in sent:
        if word in unique_words:
            encoded_sent.append(word_to_idx[word])
            counta +=1
        else:
            encoded_sent.append(0)
            count+=1
    X_dev.append(encoded_sent)
    
# print(X_dev[4])
print(count, counta)

32 6598


In [10]:
word_to_idx_tag = {}
idx_to_word_tag = {}

unique_words = set(pos_tags_total)
# print( unique_words)
for i, word in enumerate(unique_words):
    word_to_idx_tag[word] = i
    idx_to_word_tag[i] = word
    
Y = []
for sent in pos_tags:
    encoded_sent = [word_to_idx_tag[word] for word in sent]
    Y.append(encoded_sent)
    
Y_dev = []
for sent in pos_tags_dev:
    encoded_sent = [word_to_idx_tag[word] for word in sent]
    Y_dev.append(encoded_sent)

Y_test = []
for sent in pos_tags_test:
    encoded_sent = [word_to_idx_tag[word] for word in sent]
    Y_test.append(encoded_sent)

print(Y_dev[0])

[11, 6, 8, 12, 2, 5, 4, 7, 4, 7, 8, 5, 9, 2, 10, 8, 5, 2]


In [11]:
# word_to_idx = {}
# idx_to_word = {}

# unique_words_dev = set(pos_tags_total_dev)
# # print( unique_words)
# for i, word in enumerate(unique_words_dev):
#     word_to_idx[word] = i
#     idx_to_word[i] = word
# Y_dev = []
# for sent in pos_tags_dev:
#     encoded_sent = [word_to_idx[word] for word in sent]
#     Y_dev.append(encoded_sent)    

    
# print(Y_dev[0])

In [12]:
# word_to_idx = {}
# idx_to_word = {}

# unique_words_test = set(pos_tags_total_test)
# # print( unique_words)
# for i, word in enumerate(unique_words_test):
#     word_to_idx[word] = i
#     idx_to_word[i] = word
    

    
# print(Y_test[0])

In [13]:
print( idx_to_word)

{0: '1994', 1: 'november', 2: 'during', 3: 'either', 4: 'dallas', 5: 'combination', 6: 'four', 7: '1200', 8: '1026', 9: 'an', 10: 'serve', 11: 'trip', 12: 'lester', 13: 'buy', 14: 'arrival', 15: 'must', 16: 'lufthansa', 17: 'ground', 18: 'fine', 19: 'requesting', 20: '281', 21: '305', 22: 'weekdays', 23: 'jfk', 24: 'business', 25: 'co', 26: '419', 27: 'has', 28: 'fare', 29: 'have', 30: 'maximum', 31: 'abbreviations', 32: 'define', 33: 'fn', 34: '320', 35: 'nighttime', 36: 'fares', 37: 'arrange', 38: 'arrive', 39: 'today', 40: 'up', 41: '428', 42: 'land', 43: 'later', 44: 'nights', 45: 'traveling', 46: 'off', 47: 'connects', 48: 'will', 49: 'come', 50: 'mondays', 51: 'good', 52: 'else', 53: 'basis', 54: '12', 55: '1230', 56: 'thanks', 57: 'twenty', 58: '311', 59: 'air', 60: 'new', 61: 'indiana', 62: 'jet', 63: 'd9s', 64: 'round', 65: 'meaning', 66: 'like', 67: '217', 68: 'there', 69: 'dh8', 70: 'serviced', 71: '416', 72: 'more', 73: 'afterwards', 74: 'included', 75: 'am', 76: 'meals', 7

# Padding

In [14]:
length = (len(seq) for seq in X)
print(format(max(length)))

46


In [15]:
MAX_SEQ_LENGTH = 46
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_padded = pad_sequences(X,maxlen = MAX_SEQ_LENGTH, padding = 'pre', truncating = 'post' )
y_padded = pad_sequences(Y,maxlen = MAX_SEQ_LENGTH, padding = 'pre', truncating = 'post' )


In [16]:
print(x_padded[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  98 780 689 743 598 721  64  11 672 728 336 856 714
 737 631 685  57 836 733 766 631 670 404]


In [17]:
print(torch.from_numpy(np.array(x_padded[0])))

tensor([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,  98, 780, 689, 743, 598,
        721,  64,  11, 672, 728, 336, 856, 714, 737, 631, 685,  57, 836, 733,
        766, 631, 670, 404], dtype=torch.int32)


In [18]:
#  conversion done here to convert it into a proper training data set.
training_data = []
for i in range(num_sentences):
    training_data.append((X[i],Y[i]))
    
dev_data = []
for i in range(num_sentences_dev):
    dev_data.append((X_dev[i],Y_dev[i]))
    
test_data = []
for i in range(num_sentences_test):
    test_data.append((X_test[i],Y_test[i]))


In [19]:
print("d")
print(test_data[0])

d
([98, 627, 689, 176, 117, 368, 4, 733, 334, 771, 661, 418, 733, 766, 661, 0], [11, 6, 12, 5, 5, 4, 7, 10, 7, 8, 5, 2, 10, 8, 5, 9])


In [20]:
# # training sentences and their corresponding word-tags
# training_data = [
#     ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
#     ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
#     ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
# ]

# # create a dictionary that maps words to indices
# word2idx = {}

# for sent, tags in training_data:
    
#     for word in sent:
        
#         if word not in word2idx:
            
#             word2idx[word] = len(word2idx)

# # create a dictionary that maps tags to indices
# tag2idx = {"DET": 0, "NN": 1, "V": 2}

In [21]:
# # print out the created dictionary
# print(word2idx)
# print(tag2idx)


In [22]:
# import numpy as np

# # a helper function for converting a sequence of words to a Tensor of numerical values
# # will be used later in training
# def prepare_sequence(seq, to_idx):
#     '''This function takes in a sequence of words and returns a 
#     corresponding Tensor of numerical values (indices for each word).'''
    
#     idxs = [to_idx[w] for w in seq]
#     idxs = np.array(idxs)
    
#     return torch.from_numpy(idxs)

In [23]:
# # check out what prepare_sequence does for one of our training sentences:
# example_input = prepare_sequence("The dog answers the phone".lower().split(), word2idx)

# print(example_input)

# Creating the Model

In [24]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        ''' Initialize the layers of this model.'''
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        # embedding layer that turns words into a vector of a specified size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # the LSTM takes embedded word vectors (of a specified size) as inputs 
        # and outputs hidden states of size hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # the linear layer that maps the hidden state output dimension 
        # to the number of tags we want as output, tagset_size (in this case this is 3 tags)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
        # initialize the hidden state (see code below)
        self.hidden = self.init_hidden()

        
    def init_hidden(self):
        ''' At the start of training, we need to initialize a hidden state;
           there will be none because the hidden state is formed based on perviously seen data.
           So, this function defines a hidden state with all zeroes and of a specified size.'''
        # The axes dimensions are (n_layers, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        ''' Define the feedforward behavior of the model.'''
        # create embedded word vectors for each word in a sentence
        embeds = self.word_embeddings(sentence)
        
        # get the output and hidden state by passing the lstm over our word embeddings
        # the lstm takes in our embeddings and hiddent state
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        
        # get the scores for the most likely tag for a word
        tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores

# Define how the model trains 

In [25]:
# the embedding dimension defines the size of our word vectors
# for our simple vocabulary and training set, we will keep these small
EMBEDDING_DIM = 300
HIDDEN_DIM = 300

# instantiate our model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(index), num_tags)

# define our loss and optimizer
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [26]:
# test_sentence = "The cheese loves the elephant".lower().split()

# # see what the scores are before training
# # element [i,j] of the output is the *score* for tag j for word i.
# # to check the initial accuracy of our model, we don't need to train, so we use model.eval()
# inputs = prepare_sequence(test_sentence, word2idx)
# inputs = inputs
# print(inputs)
# tag_scores = model(inputs)
# print(tag_scores)

# # tag_scores outputs a vector of tag scores for each word in an inpit sentence
# # to get the most likely tag index, we grab the index with the maximum score!
# # recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
# _, predicted_tags = torch.max(tag_scores, 1)
# print('\n')
# print('Predicted tags: \n',predicted_tags)

# Train the Model

In [27]:
import random
from sklearn.metrics import classification_report

In [28]:
    label = []
    for i in range(len(idx_to_word_tag)):
        label.append(idx_to_word_tag[i])
    print(label)

['ADV', 'PART', 'ADJ', 'INTJ', 'ADP', 'NOUN', 'AUX', 'PROPN', 'VERB', 'NUM', 'CCONJ', 'PRON', 'DET']


In [29]:
n_epochs = 10
batch_size = 20

for epoch in range(n_epochs):
    
    epoch_loss = torch.tensor(0, dtype=torch.float) 
    total_length_accuracy = 0
    correct_length_accuracy = 0
    # shuffle the training data
    random.shuffle(training_data)
    batches = []
    # split the training data into batches
    for batch_start in range(0, len(training_data), batch_size):
        batch_end = batch_start + batch_size
        batch_sentences = [data[0] for data in training_data[batch_start:batch_end]]
        batch_tags = [data[1] for data in training_data[batch_start:batch_end]]
        
        batches.append(training_data[batch_start:batch_end])
#     print(batches)
    for batch in batches:
        total_loss = torch.tensor(0, dtype=torch.float) 
        
        # initialize total_loss as a PyTorch tensor object        total_length_accuracy  = 0 
#         correct_length_accuracy  = 0 
        # zero the gradients
        model.zero_grad()

        # zero the hidden state of the LSTM, this detaches it from its history
        model.hidden = model.init_hidden()
 
        # prepare the inputs for processing by out network, 
        # turn all sentences and targets into Tensors of numerical indices
        for sentence, tags in batch:
#             print(sentence)
            targets = torch.from_numpy(np.array(tags))
#             print(targets[1])
            # forward pass to get tag scores
            tag_scores = model(torch.from_numpy(np.array(sentence)))
#         print(tag_scores)
            # compute the loss, and gradients 
            loss = loss_function(tag_scores, targets)
            epoch_loss += loss
            total_loss += loss
            total_length_accuracy+=len(tag_scores)
#             tag_scores = tag_scores.item()
#             targets = targets.item()
            _, predicted_tags = torch.max(tag_scores, 1)
            for i in range(len(tag_scores)):
#                 print(predicted_tags[i], targets[i])

                if predicted_tags[i] == targets[i]:
                    correct_length_accuracy+=1
                    
#         print("necttttt")
#         print(correct_length_accuracy, total_length_accuracy)
        # compute the loss, and gradients 
        total_loss.backward()
        
        # update the model parameters with optimizer.step()
        optimizer.step()
        
#   now we have to check accuracy 
#   1. accuracy on training data
    accuracy_train = correct_length_accuracy/total_length_accuracy
    
#   2. accuracy on dev data set and test
    total_length_dev = 0 
    correct_length_dev = 0
    total_length_test = 0
    correct_length_test = 0
        
    for sentence, tags in dev_data:
        targets = torch.from_numpy(np.array(tags))
        tag_scores = model(torch.from_numpy(np.array(sentence)))
        _, predicted_tags = torch.max(tag_scores, 1)
        for i in range(len(tag_scores)):
            if predicted_tags[i] == targets[i]:
                correct_length_dev += 1
        total_length_dev += len(tag_scores)
    tags_orig = []
    tags_pred = []
    for sentence, tags in test_data:
        targets = torch.from_numpy(np.array(tags))
        for tag in tags:
            tags_orig.append(tag)
        tag_scores = model(torch.from_numpy(np.array(sentence)))
        _, predicted_tags = torch.max(tag_scores, 1)
        pred_tagu = predicted_tags.tolist()
        for tag in pred_tagu:
            tags_pred.append(tag)
        for i in range(len(tag_scores)):
            if predicted_tags[i] == targets[i]:
                correct_length_test += 1
        total_length_test += len(tag_scores)
            
        accuracy_dev = correct_length_dev/total_length_dev
        accuracy_test = correct_length_test/total_length_test
#     print(tags_orig, tags_pred)
#     label = []
#     for i in range(len(idx_to_word_tag)):
#         label.append()
    print(classification_report(tags_orig, tags_pred, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]))
    # print out avg loss per epoch
    print("Epoch: %d, loss: %1.5f, Train_Acc : %1.5f, dev_acc : %1.5f, test_acc : %1.5f" % (epoch+1, epoch_loss.item()/len(training_data),accuracy_train, accuracy_dev, accuracy_test))


              precision    recall  f1-score   support

           0       0.90      0.80      0.85        76
           1       0.98      0.96      0.97        56
           2       0.95      0.94      0.94       220
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00      1434
           5       0.99      0.99      0.99      1166
           6       0.93      0.98      0.96       256
           7       0.99      0.99      0.99      1567
           8       0.98      0.96      0.97       629
           9       0.91      0.99      0.95       127
          10       1.00      0.99      1.00       109
          11       0.96      0.76      0.85       392
          12       0.84      0.97      0.90       512

    accuracy                           0.97      6580
   macro avg       0.96      0.95      0.95      6580
weighted avg       0.97      0.97      0.97      6580

Epoch: 1, loss: 0.16160, Train_Acc : 0.94720, dev_acc : 0.96742, test_acc : 0.9

In [30]:
# Saving the model
torch.save(model.state_dict(), 'model.pt')

In [31]:
# # normally these epochs take a lot longer 
# # but with our toy data (only 3 sentences), we can do many epochs in a short time
# n_epochs = 10

# # if torch.cuda.is_available():
# #     device = torch.device("cuda")
# # else:
# #     device = torch.device("cpu")

# # model.to(device)

# for epoch in range(n_epochs):
    
#     epoch_loss = 0.0
    
#     # get all sentences and corresponding tags in the training data
#     for sentence, tags in training_data:
        
#         # zero the gradients
#         model.zero_grad()        

#         # zero the hidden state of the LSTM, this detaches it from its history
#         model.hidden = model.init_hidden()

#         # prepare the inputs for processing by out network, 
#         # turn all sentences and targets into Tensors of numerical indices
# #         sentence_in = prepare_sequence(sentence, word2idx)
#         targets = torch.from_numpy(np.array(tags))

#         # forward pass to get tag scores
#         tag_scores = model(torch.from_numpy(np.array(sentence)))
# #         print(tag_scores)
#         # compute the loss, and gradients 
#         loss = loss_function(tag_scores, targets)
#         epoch_loss += loss.item()
#         loss.backward()
        
#         # update the model parameters with optimizer.step()
#         optimizer.step()
        
#     # print out avg loss per 20 epochs
# #     if(epoch%1 == 1):
#     print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))

# Testing

In [36]:
test_sentence = "plane".lower().split()

word_to_idx = {}
idx_to_word = {}

unique_words = set(words_total)

for i, word in enumerate(unique_words):
    word_to_idx[word] = i
    idx_to_word[i] = word
    
encoded_sent = []
for word in test_sentence:
    encoded_sent.append(word_to_idx[word])
    
# print(encoded_sent)

In [37]:
# see what the scores are after training
# inputs = prepare_sequence(test_sentence, word2idx)
# inputs = inputs
tag_scores = model(torch.from_numpy(np.array(encoded_sent)))
# print(tag_scores)

# print the most likely tag index, by grabbing the index with the maximum score!
# recall that these numbers correspond to tag2idx = {"DET": 0, "NN": 1, "V": 2}
_, predicted_tagss = torch.max(tag_scores, 1)
ans = []
tags = predicted_tagss.tolist()
for idx in tags:
    encoded_sent = idx_to_word_tag[idx]
    ans.append(encoded_sent)
    
# print(idx_to_word_tag)
# print(ans)
# print('\n')
# print('Predicted tags: \n',predicted_tagss)
for i in range(len(ans)):
    print(test_sentence[i], "\t", ans[i])

plane 	 NOUN
